In [5]:
# from IPython.display import display, HTML
# display(HTML("<style>.container { width:100% !important; }</style>"))

from __future__ import division
import re
import sys
import os
import pyaudio
from google.oauth2 import service_account
from google.cloud import speech
from six.moves import queue

# Audio recording parameters
RATE = 16000
CHUNK = int(RATE / 10)  # 100ms

class MicrophoneStream(object):
    
    def __init__(self, rate, chunk):
        self._rate = rate
        self._chunk = chunk
        self._buff = queue.Queue()
        self.closed = True

    def __enter__(self):
        self._audio_interface = pyaudio.PyAudio()
        self._audio_stream = self._audio_interface.open(
            format=pyaudio.paInt16,
            channels=1,
            rate=self._rate,
            input=True,
            frames_per_buffer=self._chunk,
            stream_callback=self._fill_buffer)
        self.closed = False

        return self

    def __exit__(self, type, value, traceback):
        self._audio_stream.stop_stream()
        self._audio_stream.close()
        self.closed = True
        self._buff.put(None)
        self._audio_interface.terminate()

    def _fill_buffer(self, in_data, frame_count, time_info, status_flags):
        """Continuously collect data from the audio stream, into the buffer."""
        self._buff.put(in_data)
        return None, pyaudio.paContinue

    def generator(self):
        while not self.closed:
            chunk = self._buff.get()
            if chunk is None:
                return
            data = [chunk]
            while True:
                try:
                    chunk = self._buff.get(block=False)
                    if chunk is None:
                        return
                    data.append(chunk)
                except queue.Empty:
                    break

            yield b"".join(data)
            
def listen_print_loop(responses):
    
    for response in responses:
        if not response.results:
            continue
        result = response.results[0]
        if not result.alternatives:
            continue
        transcript = result.alternatives[0].transcript
        print(transcript)
            
def main():
    
    language_code = "en-US" 
    google_speechtotext_apikey = service_account.Credentials.from_service_account_file('*********************.json')
    client = speech.SpeechClient(credentials = google_speechtotext_apikey)
    
    diarization_config = speech.SpeakerDiarizationConfig(enable_speaker_diarization=True,
                                                         min_speaker_count=2,
                                                         max_speaker_count=3)
    
    config = speech.RecognitionConfig(encoding=speech.RecognitionConfig.AudioEncoding.LINEAR16,
                                      sample_rate_hertz=RATE,
                                      language_code=language_code,
                                      model = 'medical_conversation',
                                      diarization_config=diarization_config)
        
    streaming_config = speech.StreamingRecognitionConfig(config=config, 
                                                         interim_results=False)
    
    with MicrophoneStream(RATE, CHUNK) as stream:
        audio_generator = stream.generator()
        requests = (speech.StreamingRecognizeRequest(audio_content=content) for content in audio_generator)
        responses = client.streaming_recognize(streaming_config, requests)
        listen_print_loop(responses)
    
if __name__ == "__main__":
    
    try:
        print()
        main()
    except KeyboardInterrupt:
        print()
        print("Stopping.. as requested")


Uh , hello doctor , good morning . How are you ?
 I am fine , thank you . How you are doing ?
 So , how is your headache now
 And which medicine you took last time
 It was , uh , .
 People who have hepatitis C are more on 500 mg


Stopping.. as requested
